## Performing Mechanistic Interpretability with SDialog

In this tutorial, we will learn to use the mechanistic interpretability (M.I) feature from SDialog. We will mostly leverage the Inspector class from SDialog which is designed to be tailor-made for M.I .

This tutorial is a reproduction of : Refusal in Language Models Is Mediated by a Single Direction (https://arxiv.org/pdf/2406.11717)

Let's first define our model !

In [ ]:
import os
import torch
from typing import List
from functools import partial
from tqdm import tqdm
import warnings
from transformers.utils import logging as hf_logging
from sdialog import Turn
from sdialog.personas import Persona, PersonaAgent
hf_logging.set_verbosity_error()

MODEL_NAME = "meta-llama/Llama-3.2-3B-Instruct" # Let's define our model

We will define a base agent. For the sake of simplicty, its persona definition will remain empty.

In [ ]:
max_new_tokens = 20
persona_agent = Persona() # Let's define a blank persona for this one.

# Check if argument passing works as intended
good_agent = PersonaAgent(persona=persona_agent, model=MODEL_NAME, max_new_tokens=max_new_tokens)

Ok ! Now, we need a set of harmful and harmless instructions. We can either fetch them from external datasets (using AdvBench and Alpaca), but the theory is that we could also generate them using a third agent :)

In [ ]:
import requests
import pandas as pd
import io
from sklearn.model_selection import train_test_split

def get_harmful_instructions():
    url = 'https://raw.githubusercontent.com/SevKod/tutorials/refs/heads/main/datasets/refusal_direction/harmful_instructions.csv'
    response = requests.get(url)

    dataset = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
    instructions = dataset['goal'].tolist()

    train, test = train_test_split(instructions, test_size=0.2, random_state=42)
    return train, test

def get_harmless_instructions():
    url = 'https://raw.githubusercontent.com/SevKod/tutorials/refs/heads/main/datasets/refusal_direction/harmless_instructions.csv'
    response = requests.get(url)

    # Read CSV content
    dataset = pd.read_csv(io.StringIO(response.content.decode('utf-8')))

    # Extract instructions
    instructions = dataset['instruction'].tolist()

    # Split into train/test
    train, test = train_test_split(instructions, test_size=0.2, random_state=42)
    return train, test

harmful_inst_train, harmful_inst_test = get_harmful_instructions()
harmless_inst_train, harmless_inst_test = get_harmless_instructions()

print("Harmful instructions:")
for i in range(4):
    print(f"\t{repr(harmful_inst_train[i])}") # The spooky instructions.
print("Harmless instructions:")
for i in range(4):
    print(f"\t{repr(harmless_inst_train[i])}") # The nice ones.

harmful_inst_train = harmful_inst_train[:30]
harmless_inst_train = harmless_inst_train[:30]

Great ! We are set. Time to apply the inspector to the agent. An Inspector is a specific class designed to be attached to an agent, and allow us to observe and intervene on the internal representations of the LLM attached to the agent.

These allow us to alter the behavior of the LLM at a certain point in time.

Let's first use the Inspector to gather these internal representations and use them to compute our direction. To scope the layers of interest, we define a dictionary where the key will represent the layer we want to inspect. The value will simply be the little name we want to give to the representations we extracted.

To visualize the content of the llm, we need to access it through the agent, like this : 

In [ ]:
good_agent.llm.llm.pipeline.model

Now, finding the best layer to compute the refusal direction is not easy, we need to find the right one. Empirical studies on this specific model find layer 16 to be the best one. Let's define it in our dictionary. 

In [ ]:
from sdialog.interpretability import Inspector

layer_name_to_key_agent = {
    'model.layers.16.post_attention_layernorm': 'residual_post',
}

Good, now let's define our inspector. We can pass our dictionary like this.

In [ ]:
inspector_agent = Inspector(to_watch=layer_name_to_key_agent)

Good ! Now, the Inspector comes with a dunder-method (same as the Orchestration class), which makes it easy to attach to the agent.

In [ ]:
good_agent = good_agent | inspector_agent 

Now, suppose we attached many hooks at different places in the model, and we lost track of it. We can effectively see all the currently attached hooks through the recap() method of the inspector

In [ ]:
inspector_agent.recap()

So, we now know that the inspector has a hook attached to layer 16 ! We also know that the agent has not spoken yet. 

We also know that no instructions were found in the agent. This is really important, because the inspector also allows us to track the instruction events during generation. This allows us to effectively generate a contrast through an instruction, and find where this instruction has been applied to get the representations.

Let's stimulate our agent first.

In [ ]:
output = good_agent("Hello ! How are you ?") # Let's see how the agent responds to a simple question.
print(output)
output2 = good_agent("What is the capital of France ?") # Let's see how the agent responds to a simple question.
print(output2)

Ok good ! Now, let's look at how the inspector handles everything.

So, the inspector is a class iterable that is hierarchically decomposed in : 

[i] : i-th utterance

[i][k] : k-th token from the i-th utterance

So far, our inspector has seen only two utterances for now. We can print them.

In [ ]:
print("Output of the first utterance : ",inspector_agent[0])
print("Output of the second utterance : ",inspector_agent[1])

We can print the number of utterances as well, using len()

In [ ]:
len(inspector_agent)

Now, the inspector has a built-in hook that is automatically attached to the tokenizer of the LLM, and that pre-computes the token equivalent of the string. This means that the inspector will allow you to observe each token individually and very simply. 

We can print the first token of the first utterance.

In [ ]:
print(inspector_agent[0][0])

And, we can count the number of tokens per utterance.

In [ ]:
len(inspector_agent[0][:])

Now, remember that we previously attached one hook to observe the representations of layer 16. 

Well, we can simply fetch these representations for any token, by specifying the little value we attributed to the layer. 

In our case, it would be "residual_post".

In [ ]:
inspector_agent[0][0]['residual_post']

Great ! It works ! We have all we need to compute the direction now !

In the paper, the author gather the representations on the first token generated. If you recall, we set it to 20  to showcase the features of the Inspector. But we could set it to 1 and speed up the computation !

In [ ]:
harmful_representations = [] 
harmless_representations = []

good_agent = PersonaAgent(persona=persona_agent, model=MODEL_NAME, max_new_tokens=2)

# Harmful instructions loop
for i in tqdm(range(len(harmful_inst_train)), desc="Computing harmful representations", dynamic_ncols=True):
    good_agent(harmful_inst_train[i])
    harmful_representations.append(inspector_agent[0][0]['residual_post'])
    good_agent.reset()

# Harmless instructions loop
for i in tqdm(range(len(harmless_inst_train)), desc="Computing harmless representations", dynamic_ncols=True):
    good_agent(harmless_inst_train[i])
    harmless_representations.append(inspector_agent[0][0]['residual_post'])
    good_agent.reset()

# Concatenate and compute mean
harmful_representations = torch.cat(harmful_representations, dim=0)
harmless_representations = torch.cat(harmless_representations, dim=0)

mean_harmful = harmful_representations.mean(dim=0)
mean_harmless = harmless_representations.mean(dim=0)


We got the representations ! Now, we can compute the direction very easily by substracting the representations and normalizing them.

In [ ]:
# Get the direction
direction = mean_harmful - mean_harmless
direction = direction / direction.norm()

Ok ! We got the direction. Finally, we must steer all the layers to apply it and bypass the refusal. 

To do that, we simply need to substract the direction to the inspector. Internally, SDialog will call the function responsible for the substraction of the representations.

Again, the layers to steer must be specified when defining the inspector. Here, we will choose to steer all the layers, using our direction isolated at layer 16.

In [ ]:
layer_name_to_steer = {}
for i in range(28):
    layer_name_to_steer[f'model.layers.{i}.post_attention_layernorm'] = f'pre_mlp_{i}'
    layer_name_to_steer[f'model.layers.{i}.mlp'] = f'post_mlp_{i}'
    layer_name_to_steer[f'model.layers.{i}'] = f'residual_post_{i}'

intruder= Inspector(to_watch=layer_name_to_steer)

persona_agent = Persona()

max_new_tokens = 80

agent_steered = PersonaAgent(persona=persona_agent, model=MODEL_NAME, max_new_tokens=max_new_tokens)
agent_steered.memory = agent_steered.memory[1:] # Quick way to truncate the memory
agent_steered = agent_steered | intruder - direction

Now, we just need to use our steered agent and see how our boy now behaves with harmful request !

In [ ]:
baseline_output = []
agent_steered_output = []

nb_of_instructions_to_steer = 10

baseline_agent = PersonaAgent(persona=persona_agent, model=MODEL_NAME, max_new_tokens=max_new_tokens)

for i in tqdm(range(len(harmful_inst_test[:nb_of_instructions_to_steer])), desc="Generating harmful responses 💀:", dynamic_ncols=True):
    output = agent_steered(harmful_inst_test[i])
    agent_steered_output.append(output)
    
    output = baseline_agent(harmful_inst_test[i])
    baseline_output.append(output)

    agent_steered.reset()
    baseline_agent.reset()


In [ ]:
from colorama import Fore
import textwrap

for i in range(len(harmful_inst_test[:50])):
    print(f"INSTRUCTION {i}: {repr(harmful_inst_test[i])}")
    
    print(Fore.GREEN + "BASELINE COMPLETION:")
    print(textwrap.fill(repr(baseline_output[i]), width=100, initial_indent='\t', subsequent_indent='\t'))
    
    print(Fore.RED + "AGENT STEERED COMPLETION:")
    print(textwrap.fill(repr(agent_steered_output[i]), width=100, initial_indent='\t', subsequent_indent='\t'))
    print(Fore.RESET)


In [ ]:
agent.llm.llm.pipeline.model.model[0].post_attention_layernorm